In [1]:
import os

import numpy as np
import pandas as pd
from scipy.io import wavfile, loadmat
from scipy import signal

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision.transforms import Compose

import tensorboardX
from tqdm import tqdm

import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore') # scipy throws future warnings on fft (known bug)

In [3]:
class VerificationDatasetTrain(Dataset):
    
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        
        iden_split_path = os.path.join(path, 'iden_split.txt')
        split = pd.read_table(iden_split_path, sep=' ', header=None, names=['phase', 'path'])
        split['label'] = split['path'].apply(lambda x: int(x.split('/')[0].replace('id1', '')) - 1)
        
        # make train/test id split (in paths class id numbering starts with 1)
        fullid_arr = np.arange(1251) # 1--1251
        testid_arr = np.arange(269, 309) # 270--309
        trainid_arr = np.setdiff1d(fullid_arr, testid_arr) # 1--1251 \ 270--309
        # subsetting ids for training
        mask = split['label'].isin(trainid_arr)
        self.dataset = split['path'][mask].reset_index(drop=True)
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # path
        track_path = self.dataset[idx]
        audio_path = os.path.join(self.path, 'audio', track_path)

        # read .wav
        rate, samples = wavfile.read(audio_path)
        # extract label from path like id10003/L9_sh8msGV59/00001.txt
        # subtracting 1 because PyTorch assumes that C_i in [0, 1251-1]
        label = int(track_path.split('/')[0].replace('id1', '')) - 1

        ## parameters
        window = 'hamming'
        # window width and step size
        Tw = 25 # ms
        Ts = 10 # ms
        # frame duration (samples)
        Nw = int(rate * Tw * 1e-3)
        Ns = int(rate * (Tw - Ts) * 1e-3)
        # overlapped duration (samples)
        # 2 ** to the next pow of 2 of (Nw - 1)
        nfft = 2 ** (Nw - 1).bit_length()
        pre_emphasis = 0.97
        
        # preemphasis filter
        samples = np.append(samples[0], samples[1:] - pre_emphasis * samples[:-1])
        
        # removes DC component of the signal and add a small dither
        samples = signal.lfilter([1, -1], [1, -0.99], samples)
        dither = np.random.uniform(-1, 1, samples.shape)
        spow = np.std(samples)
        samples = samples + 1e-6 * spow * dither
        
        # segment selection
        segment_len = 3 # sec
        upper_bound = len(samples) - segment_len * rate
        start = np.random.randint(0, upper_bound)
        end = start + segment_len * rate
        samples = samples[start:end]
        
        # spectogram
        _, _, spec = signal.spectrogram(samples, rate, window, Nw, Ns, nfft, 
                                        mode='magnitude', return_onesided=False)
        
        # just multiplying it by 1600 makes spectrograms in the paper and here "the same"
        spec *= rate / 10
        
        if self.transform:
            spec = self.transform(spec)

        return label, spec

In [5]:
class VerificationDatasetTest(Dataset):
    
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        veri_split_path = os.path.join(path, 'veri_test.txt')
        self.dataset = pd.read_table(veri_split_path, sep=' ', header=None, 
                                     names=['label', 'voice1', 'voice2'])
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        # path
        label, voice1_path, voice2_path = self.dataset.iloc[idx]
        voice1_path = os.path.join(self.path, 'audio', voice1_path)
        voice2_path = os.path.join(self.path, 'audio', voice2_path)

        # read .wav
        rate, voice1 = wavfile.read(voice1_path)
        rate, voice2 = wavfile.read(voice2_path)

        ## parameters
        window = 'hamming'
        # window width and step size
        Tw = 25 # ms
        Ts = 10 # ms
        # frame duration (samples)
        Nw = int(rate * Tw * 1e-3)
        Ns = int(rate * (Tw - Ts) * 1e-3)
        # overlapped duration (samples)
        # 2 ** to the next pow of 2 of (Nw - 1)
        nfft = 2 ** (Nw - 1).bit_length()
        pre_emphasis = 0.97
        
        # preemphasis filter
        voice1 = np.append(voice1[0], voice1[1:] - pre_emphasis * voice1[:-1])
        voice2 = np.append(voice2[0], voice2[1:] - pre_emphasis * voice2[:-1])
        
        
        # removes DC component of the signal and add a small dither
        voice1 = signal.lfilter([1, -1], [1, -0.99], voice1)
        voice2 = signal.lfilter([1, -1], [1, -0.99], voice2)
        dither1 = np.random.uniform(-1, 1, voice1.shape)
        dither2 = np.random.uniform(-1, 1, voice2.shape)
        spow1 = np.std(voice1)
        spow2 = np.std(voice2)
        voice1 = voice1 + 1e-6 * spow1 * dither1
        voice2 = voice2 + 1e-6 * spow2 * dither2
        
        # spectogram
        _, _, spec1 = signal.spectrogram(voice1, rate, window, Nw, Ns, nfft, 
                                         mode='magnitude', return_onesided=False)
        _, _, spec2 = signal.spectrogram(voice2, rate, window, Nw, Ns, nfft, 
                                         mode='magnitude', return_onesided=False)
        
        # just multiplying it by 1600 makes spectrograms in the paper and here "the same"
        spec1 *= rate / 10
        spec2 *= rate / 10
        
        if self.transform:
            spec1 = self.transform(spec1)
            spec2 = self.transform(spec2)

        return label, spec1, spec2

class Normalize(object):
    """Normalizes voice spectrogram (mean-varience)"""
    
    def __call__(self, spec):
        
        # (Freq, Time)
        # mean-variance normalization for every spectrogram (not batch-wise)
        mu = spec.mean(axis=1).reshape(512, 1)
        sigma = spec.std(axis=1).reshape(512, 1)
        spec = (spec - mu) / sigma

        return spec

class ToTensor(object):
    """Convert spectogram to Tensor."""
    
    def __call__(self, spec):
        F, T = spec.shape
        
        # now specs are of size (Freq, Time) and 2D but has to be 3D (channel dim)
        spec = spec.reshape(1, F, T)
        
        # make the ndarray to be of a proper type (was float64)
        spec = spec.astype(np.float32)
        
        return torch.from_numpy(spec)

DATASET_PATH = '/home/nvme/data/vc1/'

transforms = Compose([
    Normalize(),
    ToTensor()
])

trainset = VerificationDatasetTrain(DATASET_PATH, transform=transforms)
trainsetloader = torch.utils.data.DataLoader(trainset, batch_size=3)

testset = VerificationDatasetTest(DATASET_PATH, transform=transforms)
testsetloader = torch.utils.data.DataLoader(testset, batch_size=1)

for i, a in enumerate(trainsetloader, 0):
    labels, specs = a
    print(labels, specs)
    if i > 2:
        break

for i, (label, spec1, spec2) in enumerate(testsetloader, 0):
    print(label, spec1, spec2)
    if i > 2:
        break

tensor([2, 2, 2]) tensor([[[[ 1.0483e+00,  2.1217e+00,  5.2499e-01,  ..., -4.4210e-01,
           -7.2653e-01, -2.8890e-01],
          [ 9.8662e-01,  1.7658e+00,  6.9059e-01,  ..., -5.2170e-01,
           -5.1605e-01, -7.1154e-01],
          [-4.2422e-01,  6.6823e-01, -4.6894e-01,  ..., -1.0900e+00,
            5.8806e-01,  1.5031e-01],
          ...,
          [ 9.4948e-01,  1.9095e-01,  1.2764e+00,  ..., -3.2930e-01,
            3.7034e-01, -2.4474e-01],
          [-4.2422e-01,  6.6823e-01, -4.6894e-01,  ..., -1.0900e+00,
            5.8806e-01,  1.5031e-01],
          [ 9.8662e-01,  1.7658e+00,  6.9059e-01,  ..., -5.2170e-01,
           -5.1605e-01, -7.1154e-01]]],


        [[[ 3.4684e-01,  1.6470e+00,  2.6858e+00,  ...,  1.2834e+00,
           -6.7492e-01, -5.1781e-01],
          [-2.5908e-01,  1.4524e+00,  2.8419e+00,  ...,  1.3552e+00,
           -6.7125e-01, -6.3792e-01],
          [-1.5094e+00,  6.3116e-03, -3.7809e-01,  ...,  1.5188e+00,
           -5.7417e-01,  2.9725e-01],


           -5.9822e-01, -1.8844e-01]]]]) tensor([[[[-9.3278e-01, -9.7684e-01, -5.8083e-01,  ...,  9.5456e-02,
            3.8308e+00, -9.7233e-02],
          [-9.4413e-01, -8.3841e-01, -5.6500e-01,  ..., -3.2354e-01,
            3.0884e+00,  1.4388e-01],
          [-1.0189e+00, -2.2917e-01, -4.3586e-01,  ...,  1.0657e+00,
           -5.8873e-01,  5.7299e-01],
          ...,
          [ 2.1197e-01,  2.8076e-01, -3.5094e-01,  ..., -3.2975e-01,
            6.5908e-01, -9.2398e-01],
          [-1.0189e+00, -2.2917e-01, -4.3586e-01,  ...,  1.0657e+00,
           -5.8873e-01,  5.7299e-01],
          [-9.4413e-01, -8.3841e-01, -5.6500e-01,  ..., -3.2354e-01,
            3.0884e+00,  1.4388e-01]]]])
tensor([0]) tensor([[[[-4.9645e-01, -8.4878e-01, -7.7996e-01,  ..., -7.5936e-01,
           -6.4380e-01, -5.1852e-01],
          [-3.2474e-01, -8.6533e-01, -7.4932e-01,  ..., -2.1795e-01,
           -5.9822e-01, -1.8844e-01],
          [-1.9315e-01, -9.4318e-01, -5.4747e-01,  ...,  1.0880e+00,
    